In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


In [15]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [16]:
data_dir = "E:/casper/raw_data_training"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

In [17]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)
print(device)

['IgA', 'MN']
cuda:0


In [18]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

In [19]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    best_recallmn = 0.0 # ***
    best_recallig = 0.0 # ***
    
    best_precmn = 0.0 # ***
    best_precig = 0.0 # ***
    
    best_f1mn = 0.0 # ***
    best_f1ig = 0.0 # ***

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            tp_positive = 0
            fp_positive = 0
            tn_negative = 0
            fn_negative = 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)


                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                # MN -> positive
                # iga -> negative
                tp_positive += torch.sum((preds == 1) & (labels.data == 1))
                fp_positive += torch.sum((preds == 1) & (labels.data == 0))
                tn_negative += torch.sum((preds == 0) & (labels.data == 0))
                fn_negative += torch.sum((preds == 0) & (labels.data == 1))
                    
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            epoch_recallmn = tp_positive.double() / (tp_positive + fn_negative) # ***
            epoch_recallig = tn_negative.double() / (tn_negative + fp_positive) # ***
            
            epoch_precmn = tp_positive.double() / (tp_positive + fp_positive) # ***
            epoch_precig = tn_negative.double() / (tn_negative + fn_negative) # ***
            
            epoch_f1mn = (2 * epoch_recallmn * epoch_precmn) / (epoch_recallmn + epoch_precmn) # ***
            epoch_f1ig = (2 * epoch_recallig * epoch_precig) / (epoch_recallig + epoch_precig) # ***
            
            # print('{} Loss: {:.4f} Acc: {:.4f} Recall_MGN: {:.4f} Recall_IGAN: {:.4f} Precision_MGN: {:.4f} Precision_IGAN: {:.4f} F1_MGN: {:.4f} F1_IGAN: {:.4f}'.format(
            #     phase, epoch_loss, epoch_acc, epoch_recallmn, epoch_recallig, epoch_precmn, epoch_precig, epoch_f1mn, epoch_f1ig)) # ***
            print('{} Loss: {:.4f} Accuracy: {:.4f} \n     Recall Precision F1_score\n MGN: {:.4f} {:.4f} {:.4f}\n IGAN:{:.4f} {:.4f} {:.4f}\n'.format(
                    phase, epoch_loss, epoch_acc, epoch_recallmn, epoch_precmn, epoch_f1mn, epoch_recallig, epoch_precig, epoch_f1ig)) # ***

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            if phase == 'val' and epoch_recallmn > best_recallmn: # ***
                best_recallmn = epoch_recallmn
            if phase == 'val' and epoch_recallig > best_recallig: # ***
                best_recallig = epoch_recallig  
                
            if phase == 'val' and epoch_precmn > best_precmn: # ***
                best_precmn = epoch_precmn
            if phase == 'val' and epoch_precig > best_precig: # ***
                best_precig = epoch_precig    
                
            if phase == 'val' and epoch_f1mn > best_f1mn: # ***
                best_f1mn = epoch_f1mn
            if phase == 'val' and epoch_f1ig > best_f1ig: # ***
                best_f1ig = epoch_f1ig   
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f} \n     Recall Precision F1_score\n MGN: {:.4f} {:.4f} {:.4f}\n IGAN:{:.4f} {:.4f} {:.4f}\n'.format(
                best_acc, best_recallmn, best_precmn, best_f1mn, best_recallig, best_precig, best_f1ig)) # ***

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [20]:
def train_mod(true_None, num_epoch):
    model = models.resnext101_64x4d(weights = true_None)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=num_epoch)   
    return model

In [21]:
model = train_mod(None, 2)
torch.save(model.state_dict(), "res101_2n.pt")

Epoch 0/1
----------
train Loss: 0.7461 Accuracy: 0.5755 
     Recall Precision F1_score
 MGN: 0.2913 0.3992 0.3368
 IGAN:0.7424 0.6408 0.6879

val Loss: 1.5761 Accuracy: 0.4830 
     Recall Precision F1_score
 MGN: 0.1231 0.8673 0.2156
 IGAN:0.9743 0.4487 0.6144


Epoch 1/1
----------
train Loss: 0.6968 Accuracy: 0.6155 
     Recall Precision F1_score
 MGN: 0.3669 0.4747 0.4139
 IGAN:0.7615 0.6719 0.7139

val Loss: 1.1864 Accuracy: 0.5047 
     Recall Precision F1_score
 MGN: 0.1658 0.8742 0.2788
 IGAN:0.9674 0.4593 0.6229


Training complete in 11m 9s
Best val Acc: 0.5047 
     Recall Precision F1_score
 MGN: 0.1658 0.8742 0.2788
 IGAN:0.9743 0.4593 0.6229



In [22]:
model = train_mod(True, 2)
torch.save(model.state_dict(), "res101_2t.pt")

C:\Users\VLSI\anaconda3\envs\casper_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_64X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_64X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/1
----------
train Loss: 0.6253 Accuracy: 0.6346 
     Recall Precision F1_score
 MGN: 0.0301 0.6318 0.0575
 IGAN:0.9897 0.6347 0.7734

val Loss: 0.8399 Accuracy: 0.5170 
     Recall Precision F1_score
 MGN: 0.2525 0.7390 0.3764
 IGAN:0.8782 0.4625 0.6059


Epoch 1/1
----------
train Loss: 0.5697 Accuracy: 0.6945 
     Recall Precision F1_score
 MGN: 0.3821 0.6479 0.4807
 IGAN:0.8780 0.7075 0.7836

val Loss: 0.6605 Accuracy: 0.6309 
     Recall Precision F1_score
 MGN: 0.5314 0.7567 0.6244
 IGAN:0.7667 0.5451 0.6372


Training complete in 11m 8s
Best val Acc: 0.6309 
     Recall Precision F1_score
 MGN: 0.5314 0.7567 0.6244
 IGAN:0.8782 0.5451 0.6372



In [23]:
model = train_mod(None, 5)
torch.save(model.state_dict(), "res101_5n.pt")

Epoch 0/4
----------
train Loss: 0.7434 Accuracy: 0.5676 
     Recall Precision F1_score
 MGN: 0.2851 0.3859 0.3280
 IGAN:0.7335 0.6360 0.6813

val Loss: 1.1849 Accuracy: 0.5780 
     Recall Precision F1_score
 MGN: 1.0000 0.5776 0.7323
 IGAN:0.0017 1.0000 0.0034


Epoch 1/4
----------
train Loss: 0.6907 Accuracy: 0.6188 
     Recall Precision F1_score
 MGN: 0.3726 0.4804 0.4197
 IGAN:0.7633 0.6744 0.7161

val Loss: 1.0633 Accuracy: 0.6106 
     Recall Precision F1_score
 MGN: 0.9284 0.6062 0.7335
 IGAN:0.1767 0.6438 0.2773


Epoch 2/4
----------
train Loss: 0.6654 Accuracy: 0.6345 
     Recall Precision F1_score
 MGN: 0.4252 0.5072 0.4626
 IGAN:0.7573 0.6917 0.7230

val Loss: 0.8075 Accuracy: 0.5330 
     Recall Precision F1_score
 MGN: 0.3518 0.6863 0.4651
 IGAN:0.7804 0.4686 0.5856


Epoch 3/4
----------
train Loss: 0.6461 Accuracy: 0.6525 
     Recall Precision F1_score
 MGN: 0.4508 0.5362 0.4898
 IGAN:0.7710 0.7050 0.7365

val Loss: 1.1774 Accuracy: 0.5127 
     Recall Precision F

In [24]:
model = train_mod(True, 5)
torch.save(model.state_dict(), "res101_5t.pt")

Epoch 0/4
----------
train Loss: 0.6305 Accuracy: 0.6326 
     Recall Precision F1_score
 MGN: 0.0367 0.5536 0.0689
 IGAN:0.9826 0.6346 0.7712

val Loss: 0.7566 Accuracy: 0.4743 
     Recall Precision F1_score
 MGN: 0.1369 0.7415 0.2312
 IGAN:0.9348 0.4424 0.6006


Epoch 1/4
----------
train Loss: 0.5699 Accuracy: 0.6865 
     Recall Precision F1_score
 MGN: 0.3683 0.6309 0.4651
 IGAN:0.8735 0.7019 0.7783

val Loss: 0.7119 Accuracy: 0.6229 
     Recall Precision F1_score
 MGN: 0.5063 0.7604 0.6078
 IGAN:0.7822 0.5371 0.6369


Epoch 2/4
----------
train Loss: 0.5423 Accuracy: 0.7188 
     Recall Precision F1_score
 MGN: 0.5293 0.6467 0.5821
 IGAN:0.8302 0.7502 0.7881

val Loss: 0.6425 Accuracy: 0.6606 
     Recall Precision F1_score
 MGN: 0.5905 0.7680 0.6676
 IGAN:0.7564 0.5750 0.6533


Epoch 3/4
----------
train Loss: 0.5096 Accuracy: 0.7428 
     Recall Precision F1_score
 MGN: 0.5847 0.6763 0.6272
 IGAN:0.8356 0.7741 0.8036

val Loss: 0.5888 Accuracy: 0.7208 
     Recall Precision F

In [25]:
model = train_mod(None, 20)
torch.save(model.state_dict(), "res101_20n.pt")

Epoch 0/19
----------
train Loss: 0.7502 Accuracy: 0.5719 
     Recall Precision F1_score
 MGN: 0.2802 0.3906 0.3263
 IGAN:0.7433 0.6374 0.6863

val Loss: 1.2788 Accuracy: 0.4228 
     Recall Precision F1_score
 MGN: 0.0000 nan nan
 IGAN:1.0000 0.4228 0.5943


Epoch 1/19
----------
train Loss: 0.6995 Accuracy: 0.6141 
     Recall Precision F1_score
 MGN: 0.3581 0.4717 0.4072
 IGAN:0.7644 0.6697 0.7140

val Loss: 1.3099 Accuracy: 0.4888 
     Recall Precision F1_score
 MGN: 0.1470 0.8182 0.2492
 IGAN:0.9554 0.4506 0.6124


Epoch 2/19
----------
train Loss: 0.6621 Accuracy: 0.6369 
     Recall Precision F1_score
 MGN: 0.4214 0.5114 0.4621
 IGAN:0.7635 0.6920 0.7260

val Loss: 1.1695 Accuracy: 0.4815 
     Recall Precision F1_score
 MGN: 0.1319 0.8140 0.2270
 IGAN:0.9588 0.4472 0.6099


Epoch 3/19
----------
train Loss: 0.6491 Accuracy: 0.6494 
     Recall Precision F1_score
 MGN: 0.4423 0.5316 0.4829
 IGAN:0.7711 0.7018 0.7349

val Loss: 1.2202 Accuracy: 0.4996 
     Recall Precision F1_

In [26]:
model = train_mod(True, 20)
torch.save(model.state_dict(), "res101_50t.pt")

Epoch 0/19
----------
train Loss: 0.6315 Accuracy: 0.6331 
     Recall Precision F1_score
 MGN: 0.0213 0.6250 0.0413
 IGAN:0.9925 0.6332 0.7732

val Loss: 1.0468 Accuracy: 0.4322 
     Recall Precision F1_score
 MGN: 0.0163 1.0000 0.0321
 IGAN:1.0000 0.4268 0.5983


Epoch 1/19
----------
train Loss: 0.5739 Accuracy: 0.6820 
     Recall Precision F1_score
 MGN: 0.3337 0.6334 0.4371
 IGAN:0.8865 0.6938 0.7784

val Loss: 0.9492 Accuracy: 0.6070 
     Recall Precision F1_score
 MGN: 0.4523 0.7725 0.5705
 IGAN:0.8182 0.5225 0.6377


Epoch 2/19
----------
train Loss: 0.5494 Accuracy: 0.7150 
     Recall Precision F1_score
 MGN: 0.5034 0.6478 0.5666
 IGAN:0.8392 0.7421 0.7877

val Loss: 0.8935 Accuracy: 0.6621 
     Recall Precision F1_score
 MGN: 0.6558 0.7311 0.6914
 IGAN:0.6707 0.5880 0.6266


Epoch 3/19
----------
train Loss: 0.5245 Accuracy: 0.7360 
     Recall Precision F1_score
 MGN: 0.5731 0.6667 0.6164
 IGAN:0.8317 0.7684 0.7988

val Loss: 0.7005 Accuracy: 0.6940 
     Recall Precisi

In [10]:
# model = models.resnext101_64x4d(weights = True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)
# model = model.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001)
# step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=2)

In [11]:
# #### ConvNet as fixed feature extractor ####
# # Here, we need to freeze all the network except the final layer.
# # We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward()
# model_conv = models.resnext101_64x4d(weights = True)
# for param in model_conv.parameters():
#     # print(param)
#     param.requires_grad = False

# # Parameters of newly constructed modules have requires_grad=True by default

# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 2)
# model_conv = model_conv.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that only parameters of final layer are being optimized as
# # opposed to before.
# optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# model_conv = train_model(model_conv, criterion, optimizer_conv,
#                          exp_lr_scheduler, num_epochs=2)

In [12]:
# model_v2 = models.resnext101_64x4d(weights = True)
# num_ftrs = model_v2.fc.in_features
# model_v2.fc = nn.Linear(num_ftrs, 2)
# model_v2 = model_v2.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model_v2.parameters(), lr=0.001)
# step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# model_v2 = train_model(model_v2, criterion, optimizer, step_lr_scheduler, num_epochs=20)


In [13]:
# #### ConvNet as fixed feature extractor ####
# # Here, we need to freeze all the network except the final layer.
# # We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward()
# model_conv = models.resnext101_64x4d(weights = True)
# for param in model_conv.parameters():
#     # print(param)
#     param.requires_grad = False

# # Parameters of newly constructed modules have requires_grad=True by default

# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 2)
# model_conv = model_conv.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that only parameters of final layer are being optimized as
# # opposed to before.
# optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# model_conv = train_model(model_conv, criterion, optimizer_conv,
#                          exp_lr_scheduler, num_epochs=20)